In [1]:
import torch 
import random
import numpy as np
import os
from torchvision.datasets import ImageFolder
from PIL import Image

In [2]:
# 시드값 고정

seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.enabled = False

In [3]:
#GPU 장비 설정

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [8]:
from torchvision import transforms

# 훈련, 검증, 테스트 데이터 경로 설정
train_path = 'FakeImageDetection_Dataset/train'
valid_path = 'FakeImageDetection_Dataset/valid'

class ResizeIfNeeded:
    def __call__(self, image):
        # 이미지의 가로, 세로 길이를 확인
        width, height = image.size
        
        # 가로 또는 세로 길이가 224 미만인 경우를 확인하여 크기를 늘림
        new_width = width if width >= 224 else 224
        new_height = height if height >= 224 else 224
        
        # 이미지 크기를 변경
        resized_image = image.resize((new_width, new_height), Image.BICUBIC)
        return resized_image

transform = transforms.Compose([
    # 가로 또는 세로가 224 미만일 경우 크기를 늘림
    transforms.Lambda(ResizeIfNeeded()),
    # 가로와 세로가 모두 224 이상일 경우 224x224로 랜덤 크롭
    transforms.RandomCrop((224, 224)),
    # 이미지를 PyTorch 텐서로 변환
    transforms.ToTensor(),
])

# # 훈련 데이터용 변환기
# transform_train = transforms.Compose([
#                           transforms.Resize((250, 250)),      # 이미지 크기 조정 
#                           transforms.CenterCrop(180),         # 중앙 이미지 확대
#                           transforms.RandomHorizontalFlip(0.5), # 좌우 대칭
#                           transforms.RandomVerticalFlip(0.2), # 상하 대칭
#                           transforms.RandomRotation(20),      # 이미지 회전
#                           transforms.ToTensor(),              # 텐서 객체로 변환
#                           transforms.Normalize((0.485, 0.456, 0.406), 
#                                                (0.229, 0.224, 0.225))]) # 정규화

# # 테스트 데이터용 변환기
# transform_test = transforms.Compose([
#                           transforms.Resize((250, 250)),
#                           transforms.CenterCrop(180),
#                           transforms.ToTensor(),
#                           transforms.Normalize((0.485, 0.456, 0.406), 
#                                                (0.229, 0.224, 0.225))])

# 훈련 데이터셋
datasets_train = ImageFolder(train_path, transform=transform)
# 검증 데이터셋
datasets_valid = ImageFolder(valid_path, transform=transform)

In [6]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

# 제너레이터 시드값 고정
g = torch.Generator()
g.manual_seed(0)

In [7]:
from torch.utils.data import DataLoader

batch_size = 64

loader_train = DataLoader(dataset=datasets_train, batch_size=batch_size, 
                          shuffle=True, worker_init_fn=seed_worker,
                          generator=g, num_workers=0)
loader_valid = DataLoader(dataset=datasets_valid, batch_size=batch_size, 
                          shuffle=False, worker_init_fn=seed_worker,
                          generator=g, num_workers=0)

In [7]:
from efficientnet_pytorch import EfficientNet
# 모델 생성
model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=2) 
# 장비 할당
model = model.to(device)

Loaded pretrained weights for efficientnet-b0


In [8]:
import torch.nn as nn

criterion = nn.CrossEntropyLoss()

In [9]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [10]:
from sklearn.metrics import accuracy_score # 정확도 계산 함수
from sklearn.metrics import recall_score   # 재현율 계산 함수
from sklearn.metrics import f1_score       # F1 점수 계산 함수
from tqdm.notebook import tqdm             # 진행률 표시 막대

def train(model, loader_train, loader_valid, criterion, optimizer, 
          scheduler=None, epochs=10, save_file='model_state_dict.pth'):
    
    valid_loss_min = np.inf # 최소 손실값 초기화 (검증 데이터용) 

    # 총 에폭만큼 반복
    for epoch in range(epochs):
        print(f'에폭 [{epoch+1}/{epochs}] \n-----------------------------')
        
        # == [ 훈련 ] ==============================================
        model.train()        # 모델을 훈련 상태로 설정
        epoch_train_loss = 0 # 에폭별 손실값 초기화 (훈련 데이터용)
        # '반복 횟수'만큼 반복 
        for images, labels in tqdm(loader_train):
            # 이미지, 레이블(타깃값) 데이터 미니배치를 장비에 할당 
            images = images.to(device)
            labels = labels.to(device)
            
            # 옵티마이저 내 기울기 초기화
            optimizer.zero_grad()
            # 순전파 : 이미지 데이터를 신경망 모델의 입력값으로 사용해 출력값 계산
            outputs = model(images)
            # 손실 함수를 활용해 outputs와 labels의 손실값 계산
            loss = criterion(outputs, labels)
            # 현재 배치에서의 손실 추가 (훈련 데이터용)
            epoch_train_loss += loss.item() 
            loss.backward()       # 역전파 수행
            optimizer.step()      # 가중치 갱신
            if scheduler != None: # 스케줄러 학습률 갱신 
                scheduler.step() 

        # 훈련 데이터 손실값 출력
        print(f'\t훈련 데이터 손실값 : {epoch_train_loss/len(loader_train):.4f}')
        
        # == [ 검증 ] ==============================================
        model.eval()         # 모델을 평가 상태로 설정 
        epoch_valid_loss = 0 # 에폭별 손실값 초기화 (검증 데이터용)
        preds_list = []      # 예측값 저장용 리스트 초기화
        true_list = []       # 실젯값 저장용 리스트 초기화
        
        with torch.no_grad(): # 기울기 계산 비활성화
            for images, labels in loader_valid:
                images = images.to(device)
                labels = labels.to(device)
                
                outputs = model(images)
                loss = criterion(outputs, labels)
                epoch_valid_loss += loss.item()
                
                # 예측값 및 실제값 
                preds = torch.max(outputs.cpu(), dim=1)[1].numpy() 
                true = labels.cpu().numpy() 
    
                preds_list.extend(preds)
                true_list.extend(true)
                
        # 정확도, 재현율, F1 점수 계산
        val_accuracy = accuracy_score(true_list, preds_list)
        val_recall = recall_score(true_list, preds_list)
        val_f1_score = f1_score(true_list, preds_list)

        # 검증 데이터 손실값 및 정확도, 재현율, F1점수 출력
        print(f'\t검증 데이터 손실값 : {epoch_valid_loss/len(loader_valid):.4f}')
        print(f'\t정확도 : {val_accuracy:.4f} / 재현율 : {val_recall:.4f} / F1 점수 : {val_f1_score:.4f}')
        # == [ 최적 모델 가중치 찾기 ] ==============================
        # 현 에폭에서의 손실값이 최소 손실값 이하면 모델 가중치 저장 
        if epoch_valid_loss <= valid_loss_min: 
            print(f'\t### 검증 데이터 손실값 감소 ({valid_loss_min:.4f} --> {epoch_valid_loss:.4f}). 모델 저장')
            # 모델 가중치를 파일로 저장 
            torch.save(model.state_dict(), save_file) 
            valid_loss_min = epoch_valid_loss # 최소 손실값 갱신 
    return torch.load(save_file) # 저장한 모델 가중치를 불러와 반환

In [11]:
# 모델 훈련
model_state_dict = train(model = model,
                         loader_train = loader_train, 
                         loader_valid = loader_valid,
                         criterion = criterion, 
                         optimizer = optimizer)

에폭 [1/10] 
-----------------------------


  0%|          | 0/250 [00:00<?, ?it/s]

	훈련 데이터 손실값 : 0.1001
	검증 데이터 손실값 : 0.1189
	정확도 : 0.9623 / 재현율 : 0.9505 / F1 점수 : 0.9618
	### 검증 데이터 손실값 감소 (inf --> 7.4933). 모델 저장
에폭 [2/10] 
-----------------------------


  0%|          | 0/250 [00:00<?, ?it/s]

	훈련 데이터 손실값 : 0.0319
	검증 데이터 손실값 : 0.0734
	정확도 : 0.9732 / 재현율 : 0.9465 / F1 점수 : 0.9725
	### 검증 데이터 손실값 감소 (7.4933 --> 4.6272). 모델 저장
에폭 [3/10] 
-----------------------------


...

	훈련 데이터 손실값 : 0.0086
	검증 데이터 손실값 : 0.0020
	정확도 : 0.9992 / 재현율 : 0.9990 / F1 점수 : 0.9992
	### 검증 데이터 손실값 감소 (0.3865 --> 0.1287). 모델 저장
에폭 [9/10] 
-----------------------------


  0%|          | 0/250 [00:00<?, ?it/s]

	훈련 데이터 손실값 : 0.0088
	검증 데이터 손실값 : 0.2160
	정확도 : 0.9540 / 재현율 : 0.9080 / F1 점수 : 0.9518
에폭 [10/10] 
-----------------------------


  0%|          | 0/250 [00:00<?, ?it/s]

	훈련 데이터 손실값 : 0.0092
	검증 데이터 손실값 : 0.0745
	정확도 : 0.9828 / 재현율 : 0.9655 / F1 점수 : 0.9824


In [ ]:
# 최적 가중치 불러오기
model.load_state_dict(model_state_dict)

In [ ]:
datasets_test = ImageFolder(test_path, transform=transform)

loader_test = DataLoader(dataset=datasets_test, batch_size=batch_size, 
                         shuffle=False, worker_init_fn=seed_worker,
                         generator=g, num_workers=0)

In [ ]:
def predict(model, loader_test, return_true=False):
    model.eval()    # 모델을 평가 상태로 설정
    preds_list = [] # 예측값 저장용 리스트 초기화
    true_list = []  # 실제값 저장용 리스트 초기화

    with torch.no_grad(): # 기울기 계산 비활성화
        for images, labels in loader_test:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            
            preds = torch.max(outputs.cpu(), dim=1)[1].numpy() # 예측값
            true = labels.cpu().numpy() # 실제값 

            preds_list.extend(preds)
            true_list.extend(true)

    if return_true:
        return true_list, preds_list
    else:
        return preds_list

In [ ]:
true_list, preds_list = predict(model=model, 
                                loader_test=loader_test,
                                return_true=True)

In [ ]:
print('#'*5, '최종 예측 결과 평가 점수', '#'*5)
print(f'정확도 : {accuracy_score(true_list, preds_list):.4f}')
print(f'재현율 : {recall_score(true_list, preds_list):.4f}')
print(f'F1 점수 : {f1_score(true_list, preds_list):.4f}')